<a href="https://colab.research.google.com/github/Jeremylaby/Kryptografia/blob/main/Mini%20Projekt/baby_kyber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [66]:
import numpy as np


In [81]:



class ZnW:
    def __init__(self, n, W):

        self.n = n
        self.W = np.array(W) % n

    def reduce(self, poly):
      poly = np.array(poly, dtype=int) % self.n
      while len(poly) >= len(self.W):
          factor = poly[-1]
          degree_diff = len(poly) - len(self.W)

          for i in range(len(self.W)):
              poly[degree_diff + i] -= factor * self.W[i]
          poly = poly % self.n
          while len(poly) > 0 and poly[-1] == 0:
              poly = poly[:-1]

      return poly

    def add(self, poly1, poly2):
        result = np.polyadd(poly1[::-1], poly2[::-1]) % self.n
        return self.reduce(result[::-1])

    def mul(self, poly1, poly2):
        result = np.polymul(poly1[::-1], poly2[::-1]) % self.n
        return self.reduce(result[::-1])

    def scalar_mul(self, scalar, poly):

        result = (np.array(poly) * scalar) % self.n
        return self.reduce(result)

    def __repr__(self):
        terms = []
        for i in range(len(self.W) - 1, -1, -1):  # Iteracja od najwyższej potęgi
            coeff = self.W[i]
            if coeff != 0:  # Ignoruj współczynniki zerowe
                if coeff == 1 and i != 0:  # Dla współczynnika 1 (bez "1x" lub "1x^n")
                    if i == 1:
                        terms.append("x")
                    else:
                        terms.append(f"x^{i}")
                elif coeff == -1 and i != 0:  # Dla współczynnika -1
                    if i == 1:
                        terms.append("-x")
                    else:
                        terms.append(f"-x^{i}")
                else:
                    if i == 0:
                        terms.append(f"{coeff}")
                    elif i == 1:
                        terms.append(f"{coeff}x")
                    else:
                        terms.append(f"{coeff}x^{i}")
        poly_repr = " + ".join(terms).replace("+ -", "- ") if terms else "0"  # Zmień "+ -" na "-"
        return f"ZnW({self.n}, {poly_repr})"


## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

In [82]:
q= 17
ring = ZnW(q, [1, 0, 0, 0, 1])

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

ZnW(17, 6x^3 + 16x^2 + 16x + 15)
ZnW(17, 15x^3 + 3x^2 + 10x + 2)


In [83]:
def key_gen():
  A = [
        [ [11,16,16, 6],  [3,6, 4,9] ],
        [ [ 1, 10, 3,5],  [15, 9, 1, 6] ]
    ]

  s = [
        [0, 1, 16, 16],
        [0, 16, 0, 16]
    ]

  e = [
        [0, 0, 1, 0],
        [0, 16, 1, 0]
    ]

  t = []
  for i in range(2):
      part1 = ring.mul(A[i][0], s[0])
      part2 = ring.mul(A[i][1], s[1])
      sum_  = ring.add(part1, part2)
      ti = ring.add(sum_, e[i])
      t.append(list(ti))

  return (A, t, s)
A,t,s = key_gen()
print(f'A: {A}')
print(f't: {t}')
print(f's: {s}')



A: [[[11, 16, 16, 6], [3, 6, 4, 9]], [[1, 10, 3, 5], [15, 9, 1, 6]]]
t: [[7, 0, 15, 16], [6, 11, 12, 10]]
s: [[0, 1, 16, 16], [0, 16, 0, 16]]


### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych.

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [84]:
import math
m=[1,1,0,1]
def encrypt(A,t,m):
  r=[[0,0,1,16,0],[16,0,1,1,0]]
  e1=[[0,1,1,0,0],[0,0,1,0,0]]
  e2=[0,0,16,16,0]
  u=[]
  for i in range(2):
      part1 = ring.mul(A[0][i], r[0])
      part2 = ring.mul(A[1][i], r[1])
      sum_  = ring.add(part1, part2)
      ui = ring.add(sum_, e1[i])
      u.append(list(ui))
  #print(f'u: {u}')
  part1 = ring.mul(t[0], r[0])
  part2 = ring.mul(t[1], r[1])
  sum_  = ring.add(part1, part2)
  v = ring.add(sum_, e2)
  #print(round(17 / 2))
  v = list(ring.add(v, ring.scalar_mul(9,m)))
  #print(f'v: {v}')
  return (u,v)
c = encrypt(A,t,m)
print(f'c: {c}')


c: ([[3, 10, 11, 11], [11, 13, 4, 4]], [16, 9, 6, 8])


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


In [85]:
def decrypt(c,s):
  u, v = c
  part1 = ring.mul(s[0], u[0])
  part2 = ring.mul(s[1], u[1])
  sum_su = ring.add(part1, part2)
  m_n = ring.add(v, [-v for v in sum_su])
  #print(m_n)
  m_n= [0 if x<abs(9-x) or abs(9-x) > abs(17-x)else 1 for x in m_n]
  return list(m_n)
m_n = decrypt(c,s)
print(f'm_n: {m_n}')


m_n: [1, 1, 0, 1]


### Testy

In [86]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,t,s = key_gen()

    m=[sc.choice((0,1)) for k in range(4)]

    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')


Success rate: 100.0 %
